In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from jupyter_dash import JupyterDash

import dash_html_components as html
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash

#from tqdm.notebook import tqdm
from  geopy.geocoders import Nominatim
import geopy.distance
geolocator = Nominatim(user_agent = "get_lat_lons")

import time
import datetime
import ast
import plotly.graph_objects as go
from datetime import datetime as dt


In [193]:
# Fetches CFB schedule from ESPN 

week_nums = list(range(1, 14))
cfb_games_df = pd.DataFrame(columns=["Date", "Away_Team", "Home_Team", "Stadium", "Location"])
#for week_num in tqdm(week_nums):
for week_num in week_nums:
    my_fbs_url = f'https://www.espn.com/college-football/scoreboard/_/week/{week_num}/year/2022/seasontype/2/group/80'
    time.sleep(1.5)
    page = requests.get(my_fbs_url, headers={'user-agent':
                                          'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0'})
    soup = bs(page.text, "html.parser")
    soup = soup.find_all('section', class_ ="Card gameModules")

    for sp in soup:
        # List of games per week
        games = sp.find_all('div', class_='Scoreboard__RowContainer flex flex-column flex-auto')
        # Date as container heading
        current_date = sp.find("h3").text
        for gm in games:
            away_team = gm.find_all('div', 
                                    class_='ScoreCell__TeamName ScoreCell__TeamName--shortDisplayName truncate db')[0].text
            home_team = gm.find_all('div', 
                                    class_='ScoreCell__TeamName ScoreCell__TeamName--shortDisplayName truncate db')[1].text
            stad = gm.find_all(attrs={'class':'LocationDetail__Item'})[0].text
            loc = gm.find_all(attrs={'class':'LocationDetail__Item'})[1].text
            cfb_games_df.loc[len(cfb_games_df)] = [current_date, away_team, home_team, stad, loc]

cfb_games_df['Date'] = pd.to_datetime(cfb_games_df['Date'])
cfb_games_df['League'] = 'FBS'
cfb_games_df

,Date,Away_Team,Home_Team,Stadium,Location,League
0,2022-08-27,Nevada,New Mexico State,Aggie Memorial Stadium,"Las Cruces, NM",FBS
1,2022-08-27,Austin Peay,Western Kentucky,Houchens Industries-L.T. Smith Stadium,"Bowling Green, KY",FBS
2,2022-08-27,Northwestern,Nebraska,Aviva Stadium,Dublin,FBS
3,2022-08-27,Idaho State,UNLV,Allegiant Stadium,"Las Vegas, NV",FBS
4,2022-08-27,UConn,Utah State,Maverik Stadium,"Logan, UT",FBS
...,...,...,...,...,...,...
840,2022-11-26,New Mexico,Colorado State,Canvas Stadium,"Fort Collins, CO",FBS
841,2022-11-26,UAB,Louisiana Tech,Joe Aillet Stadium,"Ruston, LA",FBS
842,2022-11-26,UTEP,UTSA,Alamodome,"San Antonio, TX",FBS
843,2022-11-26,Middle Tennessee,Florida International,Riccardo Silva Stadium,"Miami, FL",FBS


In [195]:
# Fetches NFL schedule from ESPN 

week_nums = list(range(1, 19))
nfl_games_df = pd.DataFrame(columns=["Date", "Away_Team", "Home_Team", "Stadium", "Location"])

#for week_num in tqdm(week_nums):
for week_num in week_nums:
    my_fbs_url = f'https://www.espn.com/nfl/schedule/_/week/{week_num}'
    time.sleep(1.5)
    page = requests.get(my_fbs_url, headers={'user-agent':
                                             'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0'})
    soup = bs(page.text, "html.parser")
    
    # Get all games for that week (which exist in containers separated by day)
    game_containers = soup.find('div', {'id':"sched-container"})
    dates = game_containers.find_all("h2")
    gamedays = game_containers.find_all('div', {'class':'responsive-table-wrap'})    
    for i, gameday in enumerate(gamedays):
        if i+1 > len(dates):
            break
        games = gameday.find_all('tr', {'class': ['odd', 'even']})
        for game in games:
            teams = game.findAll('a', {'class':'team-name'})
            away_team = teams[0].find('abbr')['title']
            home_team = teams[1].find('abbr')['title']
            loc_text = game.find('td', {'class':'schedule-location'}).text
            stad, city = loc_text.split(", ")
            current_date = dates[i].text
            if "January" in dates[i].text:
                current_date += ", 2023"
            else:
                current_date += ", 2022"
            nfl_games_df.loc[len(nfl_games_df)] = [current_date, away_team, home_team, stad, city]

nfl_games_df['Date'] = pd.to_datetime(nfl_games_df['Date'])

# Add State Suffix to Location
state = {'Inglewood': 'CA', 'Atlanta': 'GA', 'Chicago': 'IL', 'Cincinnati': 'OH', 'Detroit': 'MI',
         'Miami Gardens': 'FL', 'East Rutherford': 'NJ', 'Landover': 'MD', 'Charlotte': 'NC',
         'Houston': 'TX', 'Nashville': 'TN', 'Minneapolis': 'MN', 'Glendale': 'AZ', 'Arlington': 'TX',
         'Seattle': 'WA', 'Kansas City': 'MO', 'Cleveland': 'OH', 'New Orleans': 'LA', 'Pittsburgh': 'PA',
         'Jacksonville': 'FL', 'Baltimore': 'MD', 'Santa Clara': 'CA', 'Denver': 'CO', 'Las Vegas': 'NV',
         'Green Bay': 'WI', 'Orchard Park': 'NY', 'Philadelphia': 'PA', 'Indianapolis': 'IN',
         'Foxboro': 'MA', 'Tampa': 'FL', 'London': 'UK', 'Munich': 'DE', 'Mexico City': 'MX'}
nfl_games_df['Location'] = nfl_games_df['Location'].map(lambda x: x+", "+state[x])
nfl_games_df['League'] = 'NFL' 


Exception ignored in: <function tqdm.__del__ at 0x0000016218B9E200>
Traceback (most recent call last):
  File "C:\Users\daves\Documents\PythonProjects\env\lib\site-packages\tqdm\std.py", line 1162, in __del__
    self.close()
  File "C:\Users\daves\Documents\PythonProjects\env\lib\site-packages\tqdm\notebook.py", line 287, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


,Date,Away_Team,Home_Team,Stadium,Location,League
0,2022-09-08,Buffalo Bills,Los Angeles Rams,SoFi Stadium,"Inglewood, CA",NFL
1,2022-09-11,New Orleans Saints,Atlanta Falcons,Mercedes-Benz Stadium,"Atlanta, GA",NFL
2,2022-09-11,San Francisco 49ers,Chicago Bears,Soldier Field,"Chicago, IL",NFL
3,2022-09-11,Pittsburgh Steelers,Cincinnati Bengals,Paul Brown Stadium,"Cincinnati, OH",NFL
4,2022-09-11,Philadelphia Eagles,Detroit Lions,Ford Field,"Detroit, MI",NFL
...,...,...,...,...,...,...
267,2023-01-08,Cleveland Browns,Pittsburgh Steelers,Heinz Field,"Pittsburgh, PA",NFL
268,2023-01-08,Arizona Cardinals,San Francisco 49ers,Levi's Stadium,"Santa Clara, CA",NFL
269,2023-01-08,Los Angeles Rams,Seattle Seahawks,Lumen Field,"Seattle, WA",NFL
270,2023-01-08,Dallas Cowboys,Washington Commanders,FedExField,"Landover, MD",NFL


In [197]:
# Fetches FCS schedule from ESPN 

week_nums = list(range(1, 14))
fcs_games_df = pd.DataFrame(columns=["Date", "Away_Team", "Home_Team", "Stadium", "Location"])

#for week_num in tqdm(week_nums):
for week_num in week_nums:
    my_fcs_url = f'https://www.espn.com/college-football/scoreboard/_/week/{week_num}/year/2022/seasontype/2/group/81'
    time.sleep(1.5)
    page = requests.get(my_fcs_url, headers={'user-agent':
                                             'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0'})
    soup = bs(page.text, "html.parser")    
    soup = soup.find_all('section', class_ ="Card gameModules")

    for sp in soup:
        # List of games per week
        games = sp.find_all('div', class_='Scoreboard__RowContainer flex flex-column flex-auto')
        # Date as container heading
        current_date = sp.find("h3").text
        for gm in games:
            away_team = gm.find_all('div', 
                                    class_='ScoreCell__TeamName ScoreCell__TeamName--shortDisplayName truncate db')[0].text
            home_team = gm.find_all('div', 
                                    class_='ScoreCell__TeamName ScoreCell__TeamName--shortDisplayName truncate db')[1].text
            if len(gm.find_all(attrs={'class':'LocationDetail__Item'})) < 2:
                continue
            stad = gm.find_all(attrs={'class':'LocationDetail__Item'})[0].text
            loc = gm.find_all(attrs={'class':'LocationDetail__Item'})[1].text
            fcs_games_df.loc[len(fcs_games_df)] = [current_date, away_team, home_team, stad, loc]
            
            

fcs_games_df['Date'] = pd.to_datetime(fcs_games_df['Date'])
fcs_games_df['League'] = 'FCS'        

Exception ignored in: <function tqdm.__del__ at 0x0000016218B9E200>
Traceback (most recent call last):
  File "C:\Users\daves\Documents\PythonProjects\env\lib\site-packages\tqdm\std.py", line 1162, in __del__
    self.close()
  File "C:\Users\daves\Documents\PythonProjects\env\lib\site-packages\tqdm\notebook.py", line 287, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


In [198]:
all_games = pd.concat([cfb_games_df, nfl_games_df, fcs_games_df])
all_games.reset_index(drop=True, inplace=True)
all_games.drop_duplicates(subset=['Date', 'Away_Team', 'Home_Team'], keep='first', inplace=True)
all_games.drop_duplicates(subset=['Date', 'Stadium', 'Location'], keep='first', inplace=True)
all_games.drop_duplicates(subset=['Away_Team', 'Home_Team', 'Stadium'], keep='first', inplace=True)
all_games.sort_values(by=['Date'], inplace=True)

In [26]:
# Get Latitude & Longitude Measurements for Each Host Location
# There are some mistakes spotted when using the geocode module which are subsequently corrected

locations = {} 

#for loc in tqdm(all_games['Location'].unique()):
for loc in all_games['Location'].unique():
    time.sleep(1.5)
    try: 
        fetched_loc = geolocator.geocode(loc)
        locations[loc] = fetched_loc.latitude, fetched_loc.longitude
    except:
        if loc == 'San Jose, CA':
            locations[loc] = 37.320031, -121.868644
        elif loc == 'Santa Clara, CA':
            locations[loc] = 37.351505, -121.957361
        elif loc == 'Orchard Park, NY':
            locations[loc] = 42.7676, -78.7439
        elif loc == 'West Point, NY':
            locations[loc] = 41.387480, -73.964071
        else: 
            locations[loc] = "N/A"
   
all_games['Lat_Lon'] = all_games['Location'].map(lambda x: locations[x])

KeyboardInterrupt: 

In [27]:
# Output to CSV to prevent repeated scraping 

all_games.to_csv("Football_Schedule_2022.csv")

In [28]:
all_games = pd.read_csv("Football_Schedule_2022.csv", index_col=0, parse_dates=['Date'])
all_games['Lat_Lon'] = all_games.Lat_Lon.apply(lambda x: ast.literal_eval(str(x))) 

In [29]:
trips_df = pd.DataFrame(columns=['Date1','Away_Team1','Home_Team1','Stadium1','Location1','League1','Lat_Lon1',
                                 'Date2','Away_Team2','Home_Team2','Stadium2','Location2','League2','Lat_Lon2',
                                 'Date3','Away_Team3','Home_Team3','Stadium3','Location3','League3','Lat_Lon3'])
for i, j in all_games.iterrows():
    temp_df = all_games[(all_games.Date == j.Date + datetime.timedelta(days = 3)) | \
                        (all_games.Date == j.Date + datetime.timedelta(days = 2)) | \
                        (all_games.Date == j.Date + datetime.timedelta(days = 1))].copy()
    for x, y in temp_df.iterrows():
        # If both original game and game in date range are FCSvFCS games, do not include
        if (j.League == 'FCS') & (y.League == 'FCS'):
            continue
        if geopy.distance.distance(j.Lat_Lon, y.Lat_Lon).km < 150:
            trips_df.loc[len(trips_df)] = j.values.tolist() + y.values.tolist() + [np.nan]*7
            temp_df2 = all_games[(all_games.Date == y.Date + datetime.timedelta(days=2)) | \
                                 (all_games.Date == y.Date + datetime.timedelta(days=1))].copy()
            for a, b in temp_df2.iterrows():
                if (((b.League == 'FCS') & (y.League == 'FCS')) | (b.Date - j.Date > datetime.timedelta(days=5))):
                    continue
                if (geopy.distance.distance(y.Lat_Lon, b.Lat_Lon).km) < 150:
                    trips_df.loc[len(trips_df)] = j.values.tolist() + y.values.tolist() + b.values.tolist()

trips_df.to_csv("Potential_Trips.csv")

In [3]:
trips_df = pd.read_csv("Potential_Trips.csv", index_col=0)


trips_df['Date1'] = pd.to_datetime(trips_df['Date1'])
trips_df['Date2'] = pd.to_datetime(trips_df['Date2'])
trips_df['Date3'] = pd.to_datetime(trips_df['Date3'])


trips_df['Lat1'] = trips_df.Lat_Lon1.str[1:-1].str.split(", ", expand=True).astype(float)[0]
trips_df['Lon1'] = trips_df.Lat_Lon1.str[1:-1].str.split(", ", expand=True).astype(float)[1]

trips_df['Lat2'] = trips_df.Lat_Lon2.str[1:-1].str.split(", ", expand=True).astype(float)[0]
trips_df['Lon2'] = trips_df.Lat_Lon2.str[1:-1].str.split(", ", expand=True).astype(float)[1]

trips_df['Lat3'] = trips_df.Lat_Lon3.str[1:-1].str.split(", ", expand=True).astype(float)[0]
trips_df['Lon3'] = trips_df.Lat_Lon3.str[1:-1].str.split(", ", expand=True).astype(float)[1]

In [5]:
JupyterDash.infer_jupyter_proxy_config()

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
server = app.server

notes = ["No further than 150km between locations where events are held.",
         "Games no further than 3 days apart.",
         "No multiple FCS/FCS matchups in a single trip.",
         "Minor offsets introduced between locations to avoid overlap."]      

app.layout = html.Div([
    
    # Code sourced from https://github.com/Coding-with-Adam/Dash-by-Plotly/blob/master/Dash%20Components/DatePickerRange/datepicker.py
    
    dcc.DatePickerRange(
        id='my-date-picker-range',  # ID to be used for callback
        calendar_orientation='horizontal',  # vertical or horizontal
        day_size=30,  # size of calendar image. Default is 39
        end_date_placeholder_text="End Date",  # text that appears when no end date chosen
        with_portal=False,  # if True calendar will open in a full screen overlay portal
        first_day_of_week=0,  # Display of calendar when open (0 = Sunday)
        reopen_calendar_on_clear=True,
        is_RTL=False,  # True or False for direction of calendar
        clearable=False,  # whether or not the user can clear the dropdown
        number_of_months_shown=1,  # number of months shown when calendar is open
        min_date_allowed=trips_df['Date1'].min().date(),  # minimum date allowed on the DatePickerRange component
        max_date_allowed=max(trips_df['Date2'].max().date(),
                             trips_df['Date3'].max().date()),  # maximum date allowed on the DatePickerRange component
        initial_visible_month=dt(2022, 9, 1),  # the month initially presented when the user opens the calendar
        start_date=trips_df['Date1'].min().date(),
        end_date=max(trips_df['Date2'].max().date(),
                     trips_df['Date3'].max().date()),
        display_format='MMM Do, YYYY',  # how selected dates are displayed in the DatePickerRange component.
        month_format='MMM, YYYY',  # how calendar headers are displayed when the calendar is opened.
        minimum_nights=1,  # minimum number of days between start and end date

        persistence=True,
        persisted_props=['start_date', 'end_date'],
        persistence_type='session',  # session, local, or memory. Default is 'local'

        updatemode='singledate'  # singledate or bothdates. Determines when callback is triggered
    ),
    html.H3("College Football/NFL Trip Planner", style={'textAlign': 'center'}),
    html.Div(dcc.Graph(id='mymap', animate=True),
             style={"float":"center"}
            ),
    
    html.Div([
        html.Div('Notes: ', style={'font-family': 'sans-serif', 'font-weight':'bold', 'fontSize': 16}),
        html.Ul(id='notes-list', children=[html.Li(i) for i in notes])
    ])
])
  
@app.callback(
    Output('mymap', 'figure'),
    [Input('my-date-picker-range', 'start_date'),
     Input('my-date-picker-range', 'end_date')]
)



def update_output(start_date, end_date):
    
    #print(f'Start Date: {start_date}, End Date: {end_date}')
    
    trips_fltr = trips_df[(trips_df.Date1 >= start_date)&\
         (((trips_df.Date2 <= end_date)&(trips_df.Date3.isnull()))|\
         (trips_df.Date3 <= end_date))].copy().reset_index(drop=True)
    
    #print(f'Dataframe:\n{trips_fltr}')
    
    if len(trips_fltr) == 0:
        raise dash.exceptions.PreventUpdate
    
    else: 

        fig = go.Figure(go.Scattergeo())

        fig.update_geos(
            visible=False,
            scope="usa",
            showsubunits=True,
            subunitcolor="Grey",
            showframe=True,
            framecolor="Grey"

        )

        fig.update_layout(
            showlegend = False,
            extendsunburstcolors=True
        )

        for i in range(len(trips_fltr)):     
        
        # Very small offsets introduced for visibility of all points 
        
            lon_pts = [trips_fltr["Lon1"][i]*(1-np.random.random()*.001), 
                       trips_fltr["Lon2"][i]*(1-np.random.random()*.001)]
            lat_pts = [trips_fltr["Lat1"][i]*(1-np.random.random()*.001), 
                       trips_fltr["Lat2"][i]*(1-np.random.random()*.001)]


            hover_text = "(" + trips_fltr["League1"][i] + ") " + trips_fltr["Away_Team1"][i] + " @ " +\
            trips_fltr["Home_Team1"][i] +\
            " (" + trips_fltr["Date1"][i].strftime('%Y-%m-%d') + ")""<br>" +\
            "(" + trips_fltr["League2"][i] + ") " +\
            trips_fltr["Away_Team2"][i] + " @ " + trips_fltr["Home_Team2"][i] +\
            " (" + trips_fltr["Date2"][i].strftime('%Y-%m-%d') + ")" 

            if ~np.isnan(trips_fltr["Lon3"][i]):


                hover_text += "<br>" + "(" + trips_fltr["League3"][i] + ") " +\
                trips_fltr["Away_Team3"][i] + " @ " + trips_fltr["Home_Team3"][i] +\
                " (" + trips_fltr["Date3"][i].strftime('%Y-%m-%d') + ")"


                lon_pts.append(trips_fltr["Lon3"][i]*(1-np.random.random()*.001))

                lat_pts.append(trips_fltr["Lat3"][i]*(1-np.random.random()*.001))
    
            hover_text += '<extra></extra>'

            fig.add_trace(
                go.Scattergeo(
                    lon = lon_pts,
                    lat = lat_pts,
                    mode = 'markers+lines',
                    hovertemplate = hover_text,
                    marker = dict(
                        size=9.5,
                        opacity=0.55)

                    )
                )

        return fig

In [6]:
app.run_server(mode="inline")